In [1]:
%reload_ext autoreload
%autoreload 2
import colabexts
from colabexts.jcommon import *

jpath=os.path.dirname(colabexts.__file__)
jcom = f'{jpath}/jcommon.ipynb'
%run $jcom

import os, sys, datetime, re, json, importlib
from collections import defaultdict
from sys import modules
from IPython.display import HTML, Javascript
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

%matplotlib inline
pd.set_option('display.max_colwidth', 40)
pd.set_option('display.max_rows', 16)


# ESSearch.py

In [1]:
%%writefile ../map.py
class Map(dict):
    """
    Example:
    m = Map({'first_name': 'Eduardo'}, last_name='Pool', age=24, sports=['Soccer'])
    """
    def __init__(self, *args, **kwargs):
        super(Map, self).__init__(*args, **kwargs)
        for arg in args:
            if isinstance(arg, dict):
                for k, v in arg.items():
                    self[k] = v

        if kwargs:
            for k, v in kwargs.iteritems():
                self[k] = v

    def __getattr__(self, attr):
        return self.get(attr)

    def __setattr__(self, key, value):
        self.__setitem__(key, value)

    def __setitem__(self, key, value):
        super(Map, self).__setitem__(key, value)
        self.__dict__.update({key: value})

    def __delattr__(self, item):
        self.__delitem__(item)

    def __delitem__(self, key):
        super(Map, self).__delitem__(key)
        del self.__dict__[key]

#-----------------------------------------------------------------------------------
def inJupyter():
    try:    get_ipython; return True
    except: return False


Writing ../map.py


In [5]:
%%writefile ../ESsearch.py
#!/usr/local/bin/python 

#*** DO NOT EDIT - GENERATED FROM ESsearch.ipynb ****

import logging as log
import sys, os, re, math, collections, gc, urllib, elasticsearch, hashlib, pkgutil, requests
import pandas as pd
import numpy as np;
import datetime, json
from datetime import datetime, timedelta
from elasticsearch import Elasticsearch

#sys.path.append("/opt/utils")
#import filestoes, utils
from blogs.map import Map

es = Elasticsearch("http://localhost:9200/")

EXCLUDE_INDICES =["suggest_index", "PRIVATE"]
def processQ(query, DT1="1900", DT2="now/d"):
    CQ=query
    AU=""
    TI=""
    FIELDS=[]
    
    sp = [re.subn('\s+', ' ', c.strip())[0] for c in query.split(',')]
    GEN=[]
    for c in sp:
        sp1 = c.split(':')
        if (len(sp1) <=1):
            GEN.append(sp1[0])
            continue;
        sp1[0] = sp1[0].lower().strip()
        if ( sp1[0] == "author"):
            AU=sp1[1].strip()
        elif ( sp1[0] == "title"):
            TI=sp1[1].strip()
        FIELDS.append(sp1[0])
    
    
    qBody = { #MAtch and boost
          "query": {
            "function_score": { 
                "query": {
                    "bool": {
                      "should": [
                        { "match": { 
                              "query": f"{CQ}",
                        }},
                        {"multi_match": {
                          "query":    f"{CQ}",
                          "fields": FIELDS
                        }},
                        { "match": { 
                            "attachment.content":  {
                              "query": f"{CQ}",
                              "boost": 1
                        }}},
                        { "match": { 
                            "attachment.author":  {
                              "query": f"{AU}",
                              "boost": 2
                        }}},
                        { "match": { 
                            "attachment.title":  {
                              "query": f"{TI}",
                              "boost": 2
                        }}},              ],
                        #"must": { "range" : { "attachment.date" : { "gte" : DT1, "lt" :  DT2}} }
                        },
                    }},
            #"field_value_factor": { "field": "votes"}
            }}
    print("returning ", qBody)
    return qBody;
    
def SearchElastic(query, user=None, params=None, index= "", typ ="", DT1="1900", DT2="now/d"):
    print(f"Searching {index} {typ} {query} {DT1} {DT2}")
    qBody = processQ(query, DT1=DT1, DT2=DT2)
    #ls = es.search(index=INDEX_NAME, _source_exclude=['data',], filter_path=[''], q=None, body=qBody, size=25)
    ls = es.search(index=index, filter_path=[''], q=None, body=qBody, size=25)
    print(qBody, f" Returned: {ls['hits']['total']}" )
    
    # Lets also index the query into the suggest index
    #suggestindex.InsertSuggest(es, query, user)
    
    ls = Map(ls)
    return ls;

def getContext(q,cont):
    op =""
    s = [c.lower() for c in q.split()];
    cont1 = cont.lower().strip();
    
    no = ""
    for w in s:
        idx = cont1.find(w)
        if ( idx < 0 ):
            if (len(no) <= 0):
                no = "Missing: <strike>"
                
            no +=  w + "</strike> &nbsp <strike>"
            continue;
            
        t = cont[max(0, idx-64):idx+64]
        op += t + "; ..."
        if ( len(op) > 1024 ): break;
        #print("GOT : ", w, "==>", op)

    no += "</strike>"
    
    for w in q.split():
        op = re.subn(w, f"<b><i><u>{w}</u></i></b>", op, flags=re.I|re.M)[0]
        
    return (op, no)

def getType(id):
    ret = ""
    if (not id): ret =""
    elif (id.endswith("pdf"))  : ret = "pdf"
    elif (id.endswith("doc"))  : ret = "doc"
    elif (id.endswith("docx")) : ret = "doc"
    elif (id.endswith("xls"))  : ret = "xls"
    elif (id.endswith("xlsx")) : ret = "xls"
    elif (id.endswith("ppt"))  : ret = "ppt"
    elif (id.endswith("pptx")) : ret = "ppt"
    
    return ret;
    
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

def Format(q, ls):
    ret = ""
    cnt = 0
    for i,h in enumerate(ls.hits['hits']):
        source = h['_index']
        if ( source in EXCLUDE_INDICES):
            continue
        cnt += 1
        s = Map(h['_source'])
        sc= h['_score']
        a = s
        if "attachment" in s:
            a = Map(s.attachment)
        c = a.content or  ''
        d = a.date
        t = f'{a.title}]' if a.title or a.title != None else f"Content: {c[0:48]}"
        au= a.author   if 'author'  in a else ''
        ed= a.editors  if 'editors' in a else ''
        ty= getType(h['_id'].lower())
        id = h['_id'];
        did  = f"r_{i}";
        vt = s['votes'] + 2.5 if 'votes' in s else 0
        vt = 5 if vt > 5 else vt
        vt = 0 if vt < 0 else vt
        rate = f'''<span style="display: inline-block;" id="{did}" ></span>
                <script> 
                    $("#{did}").rateYo({{rating: {vt}, id:"{id}", test:76, starWidth: "16px", onSet: rateInternalDoc }}); 
                </script>
                &nbsp;|&nbsp;
                '''
        
        tit,sumy,ctx,no = "", "", '', ''
        if (c.strip()):
            ctx,no = getContext(q, c)
            sumy= c[0: max(0, 512 - len(ctx))] + f" ... <b>Context: </b>{ctx}"
            sumy = cleanhtml(sumy)
            
        it = f'''
        <a onmouseover='viewDoc("{id}")' onclick='viewdetails("{id}", "{au}", "{ed}")' class='stitle {ty}' >
            {t}
        </a>
        <a onclick='edit("{id}", "{au}", "{ed}")'> &nbsp; <i class="fas fa-edit"></i> </a>
        
        [{i}]<p class='smeta'>{d} - <b>Rev: </b> {s.rev}, Author: <b>{au} </b> ID: {id} 
        <b>[Score: </b> {sc}]</p>
        <p class=abstract>{sumy}
        <a href=# onclick="thumbsup('{id}')"><i class="fas fa-thumbs-up"></i></a>&nbsp;
        <a href# onclick="thumbsdn('{id}')"><i class="fas fa-thumbs-down"></i></a>
         
         {rate}
        <br/>
        {no}
        </p><br/>
        '''
        ret += it;
    ret1= f'''Found {cnt} items in {ls.took} ms <br/><br/>'''
    
    ret = ret1 + ret
    return ret;



Overwriting ../ESsearch.py


# ESService

In [6]:
%%writefile ../ESservice.py
#!/usr/local/bin/python 

#*** DO NOT EDIT - GENERATED FROM ESsearch.ipynb ****

import logging as log
import sys, os, re, math, datetime, json, gc, urllib, elasticsearch, hashlib, pkgutil, requests, markdown
import pandas as pd
import numpy as np;
from datetime import datetime, timedelta
from elasticsearch import Elasticsearch, helpers
from  mangorest.mango import webapi

#sys.path.append("/opt/utils")
#import filestoes
#from services.gen import suggestindex
#from services.gen import ESsearch
from blogs import suggestindex
from blogs import ESsearch

es = Elasticsearch("http://localhost:9200/")
#--------------------------------------------------------------------------------------------------------    
@webapi("/blogs/essearch")
def SearchES(q, user=None, params=None, DT1="1900", DT2="now/d", **kwargs):
    ret= ESsearch.SearchElastic(q, user, params, DT1=DT1, DT2=DT2);
    ret= ESsearch.Format(q, ret)
    
    try:
        if ( len(q) >= 3):
            suggestindex.InsertSuggestion(q, user or "unknown")
    except Exception as e:
        print("?: Suggested Index insertion Error! No worries")

    return ret
#--------------------------------------------------------------------------------------------------------    
@webapi("/essearchbyid")
def SearchESByID(ids, index='', **kwargs):
    cq = { "query": { "terms": {"_id": [f"{ids}"] }}}
          
    ls = es.search(index=index,  filter_path=[''], q=None, body=cq, size=5)
    ret = ls['hits']
    return ret;

#--------------------------------------------------------------------------------------------------------    
@webapi("/es_index_file")
def es_index_mdd(file, index='', **kwargs):
    op = filestoes.getfile(file, index)
    id = op['_source']['id']
    es.index(index=index, id=id, body=op['_source'])        
        
    return op;

#--------------------------------------------------------------------------------------------------------    
@webapi("/es_update")
def es_update(new_content, ids="", index='', **kwargs):
    body={
        "doc": {
            "content": new_content,
        }
     }
    es.update(index=index, body=body, id=ids)        
    return op;

#--------------------------------------------------------------------------------------------------------    
@webapi("/markdown")
def markit(text='', **kwargs):
    ret = markdown.markdown(text)
    #print(f'\n\n {text} ==> {ret} \n\n')
    
    return ret
#--------------------------------------------------------------------------------------------------------    
@webapi("/updatevote")
def incrementVote(id="__", score=.1, user="", index='', **kwargs):
    print( f"Ranking documents {id} to score {score} by User: {user}" )
    score = float(score)   #Score received must be b/w 0-5
    score = score - 2.5
    body={
        "script": {
            "source": "ctx._source.votes = Math.min(2.5, Math.max(0.1, ctx._source.votes + params.score))" ,
            "params": {
              "score": score
            },
            "lang": "painless"
        },
        "query": {"terms": { "_id": IDS }}
     }
    es.update_by_query(index=index, doc_type=doc_type, body=body)
    return ""
 

Overwriting ../ESservice.py


In [8]:
q="additive"
#html=SearchES(q)

ret= ESsearch.SearchElastic(q, "");
#ret= ESsearch.Format(q, ret)


Searching   additive 1900 now/d
returning  {'query': {'function_score': {'query': {'bool': {'should': [{'match': {'query': 'additive'}}, {'multi_match': {'query': 'additive', 'fields': []}}, {'match': {'attachment.content': {'query': 'additive', 'boost': 1}}}, {'match': {'attachment.author': {'query': '', 'boost': 2}}}, {'match': {'attachment.title': {'query': '', 'boost': 2}}}]}}}}}
{'query': {'function_score': {'query': {'bool': {'should': [{'match': {'query': 'additive'}}, {'multi_match': {'query': 'additive', 'fields': []}}, {'match': {'attachment.content': {'query': 'additive', 'boost': 1}}}, {'match': {'attachment.author': {'query': '', 'boost': 2}}}, {'match': {'attachment.title': {'query': '', 'boost': 2}}}]}}}}}  Returned: {'value': 5, 'relation': 'eq'}


In [9]:
ret

{'took': 9,
 'timed_out': False,
 '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 5, 'relation': 'eq'},
  'max_score': 2.4582384,
  'hits': [{'_index': 'suggest_index',
    '_id': '9082707070203273635',
    '_score': 2.4582384,
    '_source': {'count': 9,
     'timeUTC': '2022-06-17T20:08:32.407535',
     'id': 9082707070203273635,
     'title': 'additive',
     'users': {'sada.narayanappa': 5, 'snarayan': 2, 'unknown': 2},
     'timeLst': '2022-06-17T20:08:32.407538'}},
   {'_index': 'suggest_index',
    '_id': '-3603932796479293926',
    '_score': 2.4582384,
    '_source': {'title': 'additive',
     'count': 0,
     'id': -3603932796479293926,
     'users': {},
     'timeUTC': '2022-06-18T03:27:22.772146',
     'timeLst': '2022-06-18T03:27:22.772149'}},
   {'_index': 'suggest_index',
    '_id': '-5741207351631900877',
    '_score': 2.4582384,
    '_source': {'title': 'additive',
     'count': 0,
     'id': -5741207351631900877,
     '

# OTHER Testing

In [ ]:
#-----------------------------------------------------------------------------------------------
def updateAllCIJADocs(es, index='my_cija', doc_type='my_cija_type'):
    body={
        "script": {
            #"source": "ctx",
            "source": f"ctx._source.votes  = 1",
             "lang": "painless"
        },
#        "_source": False,
        "query": {"match_all": {}}
    }
    try:
        ls=es.update_by_query(index=index, doc_type=doc_type, body=body)
        return ls
    except Exception as e:
        print(e)
        return None                  

    

In [39]:
import logging as log
import sys, os, re, math, collections, gc, urllib, elasticsearch, hashlib, pkgutil, requests
import pandas as pd
import numpy as np;
import datetime, json
from datetime import datetime, timedelta
from elasticsearch import Elasticsearch


q="Boeing"

q0={ "query":{ "query_string":{ "query":"PTW"  }}}   #Simple Query String

q1 ={ #Match in specifc fields
  "query": {
    "multi_match" : {
      "query":    "PTW", 
      "fields": [ "subject", "message", "attachment.content", "attachment.title" ] 
    }
  }
}
CQ="Competitive Intelligence"
AU=""
DT1="2018-07-14"
DT2="now/d"
q3 = { #MAtch and boost
  "query": {
    "function_score": { 
        "query": {
            "bool": {
              "should": [
                { "match": { 
                      "query": f"{CQ}",
                }},
                { "match": { 
                    "attachment.content":  {
                      "query": f"{CQ}",
                      "boost": 1
                }}},
                { "match": { 
                    "attachment.author":  {
                      "query": f"{AU}",
                      "boost": 2
                }}},
                { "match": { 
                    "attachment.title":  {
                      "query": f"{CQ}",
                      "boost": 2
                }}},              
                ],
                "must": { 
                    "range" : {
                        "attachment.date" : {
                            "gte" : DT1,
                            "lt" :  DT2
                        }}},
                }},
        "field_value_factor": { 
            "field": "votes"
        }
    }
  }
}


body=q3
q=None

def esToHTML(ls):
    if ( not ls):
        return "No results"

    ret = "<TABLE><tr>"
    cols="_id,_score,votes,author,title,content,date,content_type".split(',')
    for c in cols:
        ret+= f"<th>{c}</th>"
    ret+="</tr>"
    return h
    for h in ls['hits']['hits']:
        s = h['_source'];
        vt = s['votes'] if 'votes' in s else 0
        a = s['attachment']
        au= a['author'][0:32] if 'author' in a else '?'
        ti= a['title'][0:32] if 'title' in a else '?'
        dt= a['date'][0:32] if 'date' in a else '?'

        tr = f"<tr><td>{h['_id'][0:32]}</td><td>{h['_score']}</td><td>{vt}</td>"
        tr +=f"<td>{au}</td><td>{ti}</td><td>{a['content'][0:64]}</td><td>{dt}</td><td>{a['content_type']}</td>"

        ret+= tr +"</tr>";

    ret += "</html>"

    return ret;

es = Elasticsearch()
ls=es.search(index="", q="sada", body=body);
#HTML(esToHTML(ls))
ls

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 3, 'successful': 3, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 2.6459477,
  'hits': [{'_index': 'suggest_index',
    '_type': '_doc',
    '_id': '-5201178367686124994',
    '_score': 2.6459477,
    '_source': {'title': 'sada%20',
     'count': 0,
     'id': -5201178367686124994,
     'users': {},
     'timeUTC': '2021-07-23T15:54:06.402707',
     'timeLst': '2021-07-23T15:54:06.402710'}},
   {'_index': 'internal',
    '_type': 'internal',
    '_id': '1',
    '_score': 0.2876821,
    '_source': {'rev': 1.1,
     'type': 'report',
     'author': 'Boeing Martin',
     'title': 'LM airplane better than boeing airplane',
     'topics': ['airplane', 'engine', 'lockheed', 'boeing', 'pilot'],
     'link': 'linktoactualreport',
     'content': 'we sada can put actual text from the body of the document here',
     'priority': 1}}]}}